<a href="https://colab.research.google.com/github/RahmiaWati431/KematanganTomatMenggunakanMachineLearning/blob/main/UjiNVBvsSVM70_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import display
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# --- Load Data ---
svm_crop_df = pd.read_csv('/content/drive/MyDrive/Penelitian/Berkas/Crop/70:30/svm/mcnemar/Prediksi_SVM_Crop_70:30.csv')
nb_crop_df = pd.read_csv('/content/drive/MyDrive/Penelitian/Berkas/Crop/70:30/nvb/mcnemar/Prediksi_NB_Crop_70:30.csv')

# --- Pastikan urutannya sama ---
svm_crop_df.sort_values(by='Filename', inplace=True)
nb_crop_df.sort_values(by='Filename', inplace=True)
svm_crop_df.reset_index(drop=True, inplace=True)
nb_crop_df.reset_index(drop=True, inplace=True)

# --- Pastikan label asli sama ---
assert all(svm_crop_df['Actual'] == nb_crop_df['Actual']), "⚠️ Ground truth tidak cocok!"

# --- Buat tabel evaluasi ---
evaluasi_df = pd.DataFrame({
    'Label Asli': svm_crop_df['Actual'],
    'SVM Crop Prediksi': svm_crop_df['Predicted_SVM_Crop'],
    'NB Crop Prediksi': nb_crop_df['Predicted_NB_Crop'],
})

# Tambahkan kolom kebenaran prediksi
evaluasi_df['SVM Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['SVM Crop Prediksi']
evaluasi_df['NB Benar?']  = evaluasi_df['Label Asli'] == evaluasi_df['NB Crop Prediksi']

# --- Hitung nilai a, b, c, d ---
a = sum((evaluasi_df['SVM Benar?']) & (evaluasi_df['NB Benar?']))   # Sama-sama benar
b = sum((evaluasi_df['SVM Benar?']) & (~evaluasi_df['NB Benar?']))  # SVM benar, NB salah
c = sum((~evaluasi_df['SVM Benar?']) & (evaluasi_df['NB Benar?']))  # SVM salah, NB benar
d = sum((~evaluasi_df['SVM Benar?']) & (~evaluasi_df['NB Benar?'])) # Sama-sama salah

# --- Cetak tabel kontingensi lengkap ---
table_full = [[a, b],
              [c, d]]

print("\n📋 Tabel Kontingensi McNemar (Lengkap a,b,c,d):")
print(pd.DataFrame(table_full,
                   index=['SVM Benar', 'SVM Salah'],
                   columns=['NB Benar', 'NB Salah']))

# --- Simpan tabel evaluasi ke Excel ---
output_excel = '/content/drive/MyDrive/Penelitian/Berkas/nvbvssvm/Evaluasi_McNemar_SVM_Crop_vs_NB_Crop_70-30.xlsx'
evaluasi_df.to_excel(output_excel, index=False)
print(f"\n✅ Tabel evaluasi disimpan ke: {output_excel}")

# --- Uji McNemar pakai b & c saja ---
table_bc = [[0, b],
            [c, 0]]

result = mcnemar(table_bc, exact=True)

print(f"\n📊 Hasil Uji McNemar:")
print(f"Statistik : {result.statistic}")
print(f"p-value   : {result.pvalue:.4f}")

# Interpretasi
alpha = 0.05
if result.pvalue < alpha:
    print("💡 Kesimpulan: Terdapat perbedaan signifikan antara model SVM Crop dan NB Crop (p < 0.05)")
else:
    print("💡 Kesimpulan: Tidak terdapat perbedaan signifikan antara model SVM Crop dan NB Crop (p >= 0.05)")


📋 Tabel Kontingensi McNemar (Lengkap a,b,c,d):
           NB Benar  NB Salah
SVM Benar        58         4
SVM Salah         0         1

✅ Tabel evaluasi disimpan ke: /content/drive/MyDrive/Penelitian/Berkas/nvbvssvm/Evaluasi_McNemar_SVM_Crop_vs_NB_Crop_70-30.xlsx

📊 Hasil Uji McNemar:
Statistik : 0.0
p-value   : 0.1250
💡 Kesimpulan: Tidak terdapat perbedaan signifikan antara model SVM Crop dan NB Crop (p >= 0.05)


# New Section